작업형 2
- X_train = 구매속성
- y_train = 성별 (0:여자, 1:남자)
- X_test = cust_id,구매속성 -> 성별예측 

In [1]:
# 1. 데이터불러오기
import pandas as pd
X_train = pd.read_csv("X_train.csv", encoding = 'euc-kr')
y_train = pd.read_csv("y_train.csv")
test = pd.read_csv("X_test.csv",encoding = 'euc-kr')
cust_id = test.cust_id

In [2]:
# 2. 데이터합치기
trainDF = pd.merge(X_train, y_train)

In [3]:
# 3. 결측치 확인 
trainDF.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
gender        0
dtype: int64

In [4]:
# 4. 환불금액 살펴보기
pd.DataFrame(trainDF['환불금액'])

,환불금액
0,6860000.0
1,300000.0
2,NaN
3,NaN
4,NaN
...,...
3495,NaN
3496,6049600.0
3497,NaN
3498,NaN


In [6]:
# 5. 환불금액 NAN -> 0으로 결측치 채우기 
trainDF['환불금액'] = trainDF['환불금액'].fillna(0)
# test 도 챙겨주자!
test['환불금액'] = test['환불금액'].fillna(0)
print(test)

      cust_id       총구매액     최대구매액        환불금액   주구매상품 주구매지점  내점일수    내점당구매건수  \
0        3500   70900400  22000000   4050000.0      골프  부산본점    13   1.461538   
1        3501  310533100  38558000  48034700.0     농산물   잠실점    90   2.433333   
2        3502  305264140  14825000  30521000.0    가공식품  본  점   101  14.623762   
3        3503    7594080   5225000         0.0    주방용품  부산본점     5   2.000000   
4        3504    1795790   1411200         0.0     수산품  청량리점     3   2.666667   
...       ...        ...       ...         ...     ...   ...   ...        ...   
2477     5977   82581500  23976000         0.0      골프  부산본점     8   1.750000   
2478     5978     480000    480000         0.0    섬유잡화   광주점     1   1.000000   
2479     5979  260003790  25750000         0.0  남성 캐주얼  본  점    19   3.736842   
2480     5980   88991520  18120000         0.0      육류  본  점     5   3.600000   
2481     5981     623700    209000         0.0    가공식품  영등포점     2   5.000000   

        주말방문비율  구매주기  
0   

In [82]:
# 6. 주구매상품, 주구매지점 범주형 변수 -> LabelEncoder 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainDF.loc[:,['주구매상품','주구매지점']] = trainDF.loc[:,['주구매상품','주구매지점']].apply(le.fit_transform)
# test 도 챙겨주자! 
test.loc[:,['주구매상품','주구매지점']] = test.loc[:,['주구매상품','주구매지점']].apply(le.fit_transform)

In [83]:
# 7. 합친 데이터를 X, y로 다시 나누자
X = trainDF.drop(['cust_id','gender'], axis=1) # axis=1 안해주면 에러남. 열(column)을 드랍한다고 알려주기
y = pd.DataFrame(trainDF.gender) # series로 되는게 싫어서 애초에 데이터 프레임으로 바꿈 

In [84]:
# 8. train test 나눠주자 (? 원래 있던 X_test는 마지막에 어떻게 검증하는지 모르겠음)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)

In [90]:
# 9. 정규화 - MinMax/StandardScaler 중 선택 둘다해서 결과비교해보기
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)
test_scaled = scaler.transform(test)

In [86]:
# 10. 모델적용 RandomForestClassifier 분류문제니까
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

/Users/yeona/opt/anaconda3/envs/pydatavenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.9992380952380953

In [87]:
# X_scaled_test, y_test 챙겨주자
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.6331428571428571

In [104]:
# 결과를 보면 train 데이터에 과적합 되었다는걸 알 수 있다.
#test = test.drop(['cust_id'],axis=1)
predict_gender = pd.DataFrame(model.predict_proba(test_scaled)[:,1], columns=['gender'])
predict_gender

,gender
0,0.50
1,0.25
2,0.23
3,0.41
4,0.35
...,...
2477,0.66
2478,0.51
2479,0.52
2480,0.43


In [113]:
# 제출할 데이터 합치기 
result = pd.concat([cust_id,predict_gender], axis=1)
result.head()

,cust_id,gender
0,3500,0.50
1,3501,0.25
2,3502,0.23
3,3503,0.41
4,3504,0.35


In [111]:
pd.read_csv("git/DataAnalysis/RF.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'git/DataAnalysis/RF.csv'

In [97]:
type(predict_gender)

numpy.ndarray